In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from shapely.geometry import Point,Polygon,LineString
import cv2 as cv
import glob
import pandas as pd
import os

In [ ]:
#Returns f(x)
def f(x,a,b) : 
    y = a * x + b  
    return y

In [ ]:
#Rotation matrix with an angle of alpha
def rotation(alpha) :
    return np.array([[np.cos(alpha), np.sin(alpha)],[-np.sin(alpha), np.cos(alpha)]])

In [ ]:
#Region creation in function of the frame dimensions and the diagonals
def Regions(W,H,Hips,rightline,leftline,keypoints,ndh_type) :
    
    #R1 & R2 
    if(rightline[-1,0] == W) : #If the blue diagonal is touching the upper x-limit of the frame
        R1 = [(Hips[0], Hips[1]), (Hips[0], 0), (W, 0), (W, rightline[-1,1])]
        R2 = [(Hips[0], Hips[1]), (W, rightline[-1,1]), (W,Hips[1])]
        
    elif(rightline[-1,1] == 0) : #If the blue diagonal is touching the lower y-limit of the frame
        R1 = [(Hips[0], Hips[1]), (Hips[0], 0), (rightline[-1,0], 0)]
        R2 = [(Hips[0], Hips[1]), (rightline[-1,0], 0), (W, 0), (W,Hips[1])]
        
    else : #If the blue diagonal is exactly going through the right-hand upper corner of the frame
        R1 = [(Hips[0], Hips[1]), (Hips[0], 0), (W, 0)]
        R2 = [(Hips[0], Hips[1]), (W, 0),(W,Hips[1])]

        
    #R3 & R4    
    if(leftline[-1,0] == W) : #If the red diagonal is touching the upper x-limit of the frame
        R3 = [(Hips[0], Hips[1]), (W,Hips[1]), (W,leftline[-1,1])]
        R4 = [(Hips[0], Hips[1]), (W,leftline[-1,1]), (W,H), (Hips[0],H)]
        
    elif(leftline[-1,1] == H) : #If the red diagonal is touching the upper y-limit of the frame
        R3 = [(Hips[0], Hips[1]), (W,Hips[1]), (W,H), (leftline[-1,0],H)]
        R4 = [(Hips[0], Hips[1]), (leftline[-1,0],H), (Hips[0],H)]
        
    else : #If the red diagonal is exactly going through the right-hand lower corner of the frame
        R3 = [(Hips[0], Hips[1]), (W,Hips[1]), (W,H)]
        R4 = [(Hips[0], Hips[1]), (W,H), (Hips[0],H)]

        
    #R5 & R6    
    if(rightline[0,0] == 0) : #If the blue diagonal is touching the lower x-limit of the frame
        R5 = [(Hips[0], Hips[1]), (Hips[0],H), (0,H), (0, rightline[0,1])]
        R6 = [(Hips[0], Hips[1]), (0, rightline[0,1]), (0,Hips[1])]
        
    elif(rightline[0,1] == H) : #If the blue diagonal is touching the upper y-limit of the frame
        R5 = [(Hips[0], Hips[1]), (Hips[0],H), (rightline[0,0],H)]
        R6 = [(Hips[0], Hips[1]), (rightline[0,0],H), (0,H), (0,Hips[1])]
        
    else : #If the blue diagonal is exactly going through the left-hand lower corner of the frame
        R5 = [(Hips[0], Hips[1]), (Hips[0],H), (0,H)]
        R6 = [(Hips[0], Hips[1]), (0,H), (0,Hips[1])]

        
    #R7 & R8   
    if(leftline[0,0] == 0) : #If the red diagonal is touching the lower x-limit of the frame
        R7 = [(Hips[0], Hips[1]), (0,Hips[1]), (0,leftline[0,1])]
        R8 = [(Hips[0], Hips[1]), (0,leftline[0,1]), (0,0), (Hips[0], 0)]

    elif(leftline[0,1] == 0) : #If the red diagonal is touching the lower y-limit of the frame
        R7 = [(Hips[0], Hips[1]), (0,Hips[1]), (0,0), (leftline[0,0],0)]
        R8 = [(Hips[0], Hips[1]), (leftline[0,0],0), (Hips[0], 0)]
        
    else : #If the red diagonal is exactly going through the left-hand upper corner of the frame
        R7 = [(Hips[0], Hips[1]), (0,Hips[1]), (0,0)]
        R8 = [(Hips[0], Hips[1]), (0,0), (Hips[0], 0)]

    #Creation of the polygons composed of the points defined earlier
    poly_R1 = Polygon(R1)
    poly_R2 = Polygon(R2)
    poly_R3 = Polygon(R3)
    poly_R4 = Polygon(R4)
    poly_R5 = Polygon(R5)
    poly_R6 = Polygon(R6)
    poly_R7 = Polygon(R7)
    poly_R8 = Polygon(R8)

    #Matrix containing the keypoints in each region : size = (number of regions, number of keypoints)
    R_key = np.zeros((8,len(keypoints[:,:])))
    
    #For every keypoint, we check in which polygon(region) they are contained
    for i in range(len(keypoints[:,:])) :
        if(keypoints[i,0] != 0 or keypoints[i,1] != 0) :
            target = Point(keypoints[i,0], keypoints[i,1])
            R_key[0,i] = poly_R1.contains(target)
            R_key[1,i] = poly_R2.contains(target)
            R_key[2,i] = poly_R3.contains(target)
            R_key[3,i] = poly_R4.contains(target) 
            R_key[4,i] = poly_R5.contains(target)
            R_key[5,i] = poly_R6.contains(target)
            R_key[6,i] = poly_R7.contains(target)
            R_key[7,i] = poly_R8.contains(target)

            
    #print(R_key)
    #print(sum(R_key))
    #print(np.sum(R_key, axis = 1))
    if ndh_type == 'ndh' :
    #The final NDH is the sum of the columns of the global region matrix
        return np.sum(R_key, axis = 1)
    else :
        return R_key.reshape(1,8*len(keypoints[:,:]))

In [ ]:
def NDH(keypoints,W,H,ndh_type) : 
        
    x = np.linspace(0,W,num = 500)
    y = np.linspace(0,H,num = 500)
    
    #NDH matrix containing (id,NDH) per human : size = 
                                #(number of humans, len(id) + len(region matrix)) = (number of humans, 9) (if image)
                                #(number of frames for the tragetted human, 9) (if video)
    
    Hips = np.mean(np.concatenate((keypoints[8,:].reshape(1,2),keypoints[11,:].reshape(1,2)),axis = 0),axis=0).reshape(1,2)[0]
    
    #Diagonals computation
    leftline = rotation(np.radians(45))@np.array([(Hips[0]*np.ones(len(x)) - Hips[0]),(y - Hips[1])]) + [np.ones(len(x))*Hips[0],np.ones(len(x))*Hips[1]]
    rightline = rotation(np.radians(-45))@np.array([(Hips[0]*np.ones(len(x)) - Hips[0]),(y - Hips[1])]) + [np.ones(len(x))*Hips[0],np.ones(len(x))*Hips[1]]
    coeff_left = np.polyfit(leftline[0], leftline[1], 1)
    coeff_right = np.polyfit(rightline[0], rightline[1], 1)

    leftline = np.concatenate((x.reshape(-1,1),f(x,coeff_left[0],coeff_left[1]).reshape(-1,1)),axis = 1)
    rightline =  np.concatenate((x.reshape(-1,1),f(x,coeff_right[0],coeff_right[1]).reshape(-1,1)), axis = 1)

    #NDH for every human in the frame
    ndh = Regions(W,H,Hips,rightline,leftline,keypoints,ndh_type)
        
    return ndh

In [ ]:
def Display_img(frame,keypoints,label) :    
    #Display of the keypoints, skeleton and bounding box of each human
    img = cv.imread(frame)

    thickness = 1
    color_skeleton = (255, 180, 90)
    color = (0,255,0)
    color_box = (255,255,255)

    #BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4, "LShoulder": 5, 
                  #"LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9, "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13}
    
    
    LINES_BODY = [[0,1],[1,2],[2,3],[3,4],[1,5],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13]]

    #Skeleton
    lines = [np.array([np.int64(keypoints[point,:]) for point in line]) for line in LINES_BODY]
    frame_keypoints = cv.polylines(img, lines, False, color_skeleton, 1, cv.LINE_AA)

    #Keypoints
    for j in range(len(keypoints)):       
        cv.circle(img, (np.int64(keypoints[j,0]), np.int64(keypoints[j,1])), 2, color, -11)

    cv.putText(img, frame.rsplit('/')[2] + '/' + frame.rsplit('/')[3], (20,20), cv.FONT_HERSHEY_SIMPLEX, 0.5, color_box, 1)
    cv.putText(img, 'label : ' + str(label), (20, 40), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
    
    #cv.imwrite('GIF/'+frame.split('/')[-1], img)
    cv.imshow("output",img)
    cv.waitKey(0)
    cv.destroyAllWindows() 

# I. Frame annotations processing

## 1) Scenario dataset

In [ ]:
sc_number = 3  #1,2,3,5,6,7,10

keypoints_dataframe = pd.DataFrame()

for i in range(1,9) :
    
    path = "DATA/SC"+str(sc_number)+"_Frames_annot/SF_cam"+str(i)+"_sc"+str(sc_number)+""

    files = glob.glob(path + "/*.csv")

    for filename in files:
        df = pd.read_csv(filename, index_col=None)
        df = pd.DataFrame(df)
        
        video_name = "SF_cam"+str(i)+"_sc"+str(sc_number)+""
        video = [video_name] * len(df)
        df['Video'] = video
        
    frame = [keypoints_dataframe, df]
    keypoints_dataframe = pd.concat(frame,axis = 0)
    
#with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
keypoints_dataframe = keypoints_dataframe.set_index('Video')
keypoints_dataframe

### Sort for every frame number in each video

In [ ]:
for i in range(1,9) :
    set_KP = keypoints_dataframe.loc['SF_cam' + str(i) + '_sc'+str(sc_number)]
    set_KP['frame2'] = set_KP['frame'].str.split('e',expand=True)[1]
    set_KP['frame2'] = set_KP['frame2'].str.split('.',expand=True)[0].astype(int)
    set_KP = set_KP.sort_values(by = ['frame2','KP'])
    del set_KP['frame2']
    keypoints_dataframe.loc['SF_cam' + str(i) + '_sc'+str(sc_number)] = set_KP
    
print(keypoints_dataframe)

### Save keypoints

In [ ]:
keypoints_dataframe.to_csv('DATA/SC'+str(sc_number)+'_Frames_annot/SC'+str(sc_number)+'_Frames_annot.csv')
#keypoints_dataframe = pd.read_csv("DATA/SC"+str(sc_number)+"_Frames_annot/SC"+str(sc_number)+"_Frames_annot.csv",index_col = 0)
keypoints_dataframe

## 2) Homemade dataset

In [ ]:
keypoints_dataframe = pd.DataFrame()
content = []

for fall_number in range(3,13) :
    
    df = pd.read_csv("DATA/Falls_6frames/Fall_"+str(fall_number)+"/Fall_"+str(fall_number)+"_KP.csv")
    df = pd.DataFrame(df)

    video_name = "Fall_"+str(fall_number)
    video = [video_name] * len(df)
    df['Video'] = video

    frame = [keypoints_dataframe, df]
    keypoints_dataframe = pd.concat(frame,axis = 0)
    content = []
    
#with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
keypoints_dataframe = keypoints_dataframe.set_index('Video')
del keypoints_dataframe['KP_name']
with pd.option_context('display.max_rows', None, 'display.max_columns', None) :
    print(keypoints_dataframe)

In [ ]:
for fall_number in range(3,13) :
    set_KP = keypoints_dataframe.loc['Fall_'+str(fall_number)]
    set_KP['frame2'] = set_KP['frame'].str.split(str(fall_number)+"_",expand=True)[1]
    set_KP['frame2'] = set_KP['frame2'].str.split('.',expand=True)[0].astype(int)
    set_KP = set_KP.sort_values(by = ['frame2','KP'])
    del set_KP['frame2']
    keypoints_dataframe.loc['Fall_'+str(fall_number)] = set_KP
    
keypoints_dataframe

### Save keypoints

In [ ]:
keypoints_dataframe.to_csv('DATA/Falls_6frames/Falls_Frames_annot.csv')
keypoints_dataframe

# II. Label processing

   ## 1) Fall scenarios dataset

In [ ]:
sc_number = 3  #1,2,3,5,6,7,10

labels_dataframe = pd.DataFrame()
content = []
    
path = "DATA/SC"+str(sc_number)+"_Frames_annot/sc"+str(sc_number)+"_labels"

files = glob.glob(path + "/*.csv")

for filename in files:
    df = pd.read_csv(filename, sep = ',', index_col=None)
    
    video_name = os.path.basename(filename).rsplit('.', 1)[0]
    video = [video_name] * len(df)
    df['Video'] = video
    
    content.append(df)    

labels_dataframe = pd.concat(content)

#with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
labels_dataframe = labels_dataframe.set_index('Video')
#print(labels_dataframe) 

In [ ]:
#trier en fonction de frame number
for i in range(1,9) :
    set_label = labels_dataframe.loc['cam' + str(i) + '_sc'+str(sc_number)+'_labels']
    set_label['frame2'] = set_label['frame'].str.split('e',expand=True)[1]
    set_label['frame2'] = set_label['frame2'].str.split('.',expand=True)[0].astype(int)
    set_label = set_label.sort_values(by = ['frame2'])
    del set_label['frame2']
    
    set_label['label2'] = set_label['label'].str.split('[',expand=True)[1]
    set_label['label2'] = set_label['label2'].str.split(']',expand=True)[0]
    set_label['label'] = set_label['label2']
    del set_label['label2']
    
    labels_dataframe.loc['cam' + str(i) + '_sc'+str(sc_number)+'_labels'] = set_label

In [ ]:
labels_dataframe
labels_dataframe.to_csv('DATA/SC'+str(sc_number)+'_Frames_annot/SC'+str(sc_number)+'_Labels.csv')

## 2) Homemade dataset

In [ ]:
labels_dataframe = pd.DataFrame()
content = []

for fall_number in range(3,13) :

    df = pd.read_csv("DATA/Falls_6frames/Fall_"+str(fall_number)+"/Fall_"+str(fall_number)+"_labels.csv")
    df = pd.DataFrame(df)

    video_name = "Fall_"+str(fall_number)
    video = [video_name] * len(df)
    df['Video'] = video
    
    frame = [labels_dataframe, df]
    labels_dataframe = pd.concat(frame,axis = 0)
    content = []
    
#with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
labels_dataframe = labels_dataframe.set_index('Video')
labels_dataframe

In [ ]:
for fall_number in range(3,13) :
    set_label = labels_dataframe.loc['Fall_'+str(fall_number)]
    set_label['frame2'] = set_label['frame'].str.split(str(fall_number)+"_",expand=True)[1]
    set_label['frame2'] = set_label['frame2'].str.split('.',expand=True)[0].astype(int)
    set_label = set_label.sort_values(by = ['frame2'])
    del set_label['frame2']
    
    set_label['label2'] = set_label['label'].str.split('[',expand=True)[1]
    set_label['label2'] = set_label['label2'].str.split(']',expand=True)[0]
    set_label['label'] = set_label['label2']
    del set_label['label2']
    
    labels_dataframe.loc['Fall_'+str(fall_number)] = set_label

In [ ]:
labels_dataframe
labels_dataframe.to_csv('DATA/Falls_6frames/Falls_Labels.csv')

# III. NDH application
## 1) Fall scenarios dataset

In [ ]:
ndh = np.zeros((np.int64(len(keypoints_dataframe)/14),8))
ii = 0
for i in range(1,9) :
    KP = keypoints_dataframe.loc['SF_cam' + str(i) + '_sc'+str(sc_number)][['KP','x','y','W','H','frame']]
    size = len(KP)
    
    keypoints = np.zeros((np.int64(size/14),14,2))
    W = np.zeros((np.int64(size/14)))
    H = np.zeros((np.int64(size/14)))
    
    for j in range(0,np.int64(size/14)):
        KP2 = KP.iloc[j*14:j*14+14]
        keypoints[j,:,:] = KP2[['x','y']].to_numpy()
        W[j] = KP2['W'][j]
        H[j] = KP2['H'][j]
        
        ndh[ii,:] = NDH(keypoints[j,:,:],W[j],H[j],'ndh')
        frame = KP2['frame'][j]
        frame = 'DATA/SC'+str(sc_number)+'_Frames_annot/SF_cam' + str(i) + '_sc'+str(sc_number)+'/'+frame
        #Display_img(frame,keypoints[j,:,:],labels_dataframe['label'].iloc[ii])
        ii += 1

In [ ]:
print(ndh.shape)

In [ ]:
np.savetxt('DATA/SC'+str(sc_number)+'_Frames_annot/SC'+str(sc_number)+'_NDH.csv', ndh, delimiter=",")

## 2) Homemade dataset

In [ ]:
keypoints_dataframe = pd.read_csv("DATA/Falls_6frames/Falls_Frames_annot.csv", index_col = 0)
labels_dataframe = pd.read_csv("DATA/Falls_6frames/Falls_Labels.csv", index_col = 0)

ndh = np.zeros((np.int64(len(keypoints_dataframe)/14),8))
ii = 0

for fall_number in range(3,13) : #3,13
    KP = keypoints_dataframe.loc['Fall_'+str(fall_number)][['KP','x','y','W','H','frame']]
    size = len(KP)
    
    keypoints = np.zeros((np.int64(size/14),14,2))
    W = np.zeros((np.int64(size/14)))
    H = np.zeros((np.int64(size/14)))
    
    for j in range(0,np.int64(size/14)):
        KP2 = KP.iloc[j*14:j*14+14]
        #print(labels_dataframe['label'].iloc[ii])
        keypoints[j,:,:] = KP2[['x','y']].to_numpy()
        W[j] = KP2['W'][0]
        H[j] = KP2['H'][0]
        
        ndh[ii,:] = NDH(keypoints[j,:,:],W[j],H[j],'ndh')
        frame = KP2['frame'][0]
        frame = 'DATA/Falls/Fall_'+str(fall_number)+'/'+frame
        #Display_img(frame,keypoints[j,:,:],labels_dataframe['label'].iloc[ii])
        ii += 1

In [ ]:
#print(ndh)
np.savetxt('DATA/Falls_6frames/Falls_NDH.csv', ndh, delimiter=",")

# IV. RegKey application

## 1) Fall scenarios dataset

In [ ]:
sc_list = [1,2,3,5,6,7,10]

for sc_number in sc_list :
    keypoints_dataframe = pd.read_csv("DATA/SC"+str(sc_number)+"_Frames_annot/SC"+str(sc_number)+"_Frames_annot.csv",index_col = 0)
    labels_dataframe = pd.read_csv("DATA/SC"+str(sc_number)+"_Frames_annot/SC"+str(sc_number)+"_Labels.csv",index_col = 0)
    ndh = np.zeros((np.int64(len(keypoints_dataframe)/14),8*14))
    ii = 0
    for i in range(1,9) :
        KP = keypoints_dataframe.loc['SF_cam' + str(i) + '_sc'+str(sc_number)][['KP','x','y','W','H','frame']]
        size = len(KP)

        keypoints = np.zeros((np.int64(size/14),14,2))
        W = np.zeros((np.int64(size/14)))
        H = np.zeros((np.int64(size/14)))

        for j in range(0,np.int64(size/14)):
            KP2 = KP.iloc[j*14:j*14+14]
            keypoints[j,:,:] = KP2[['x','y']].to_numpy()
            W[j] = KP2['W'][j]
            H[j] = KP2['H'][j]

            ndh[ii,:] = NDH(keypoints[j,:,:],W[j],H[j],'RegKey')
            frame = KP2['frame'][j]
            frame = 'DATA/SC'+str(sc_number)+'_Frames_annot/SF_cam' + str(i) + '_sc'+str(sc_number)+'/'+frame
            #Display_img(frame,keypoints[j,:,:],labels_dataframe['label'].iloc[ii])
            ii += 1
    np.savetxt('DATA/SC'+str(sc_number)+'_Frames_annot/SC'+str(sc_number)+'_RegKey.csv', ndh, delimiter=",")

## 2) Homemade dataset

In [ ]:
keypoints_dataframe = pd.read_csv("DATA/Falls/Falls_Frames_annot.csv", index_col = 0)
labels_dataframe = pd.read_csv("DATA/Falls/Falls_Labels.csv", index_col = 0)

ndh = np.zeros((np.int64(len(keypoints_dataframe)/14),8*14))
ii = 0
REGKEY = pd.DataFrame()


for fall_number in range(3,13) : #3,13
    KP = keypoints_dataframe.loc['Fall_'+str(fall_number)][['KP','x','y','W','H','frame']]
    size = len(KP)
    
    keypoints = np.zeros((np.int64(size/14),14,2))
    W = np.zeros((np.int64(size/14)))
    H = np.zeros((np.int64(size/14)))
    
    for j in range(0,np.int64(size/14)):
        KP2 = KP.iloc[j*14:j*14+14]
        #print(labels_dataframe['label'].iloc[ii])
        keypoints[j,:,:] = KP2[['x','y']].to_numpy()
        W[j] = KP2['W'][0]
        H[j] = KP2['H'][0]
        
        ndh[ii,:] = NDH(keypoints[j,:,:],W[j],H[j],'RegKey')
        frame = KP2['frame'][0]
        frame = 'DATA/Falls/Fall_'+str(fall_number)+'/'+frame
        #Display_img(frame,keypoints[j,:,:],labels_dataframe['label'].iloc[ii])
        ii += 1
    
    #print(ndh[ii-np.int64(size/14) :ii,:])
    
    df = pd.DataFrame(ndh[ii-np.int64(size/14) :ii,:])
    video_name = "Fall_" + str(fall_number) 
    video = [video_name] * len(df)
    df['Video'] = video
                
    frame = [REGKEY, df]
    REGKEY = pd.concat(frame,axis = 0)

In [ ]:
REGKEY = REGKEY.set_index('Video')
REGKEY

In [ ]:
np.savetxt('DATA/Falls/Falls_RegKey.csv', ndh, delimiter=",")
REGKEY.to_csv('DATA/Falls/Falls_RegKey_DataFrame.csv')

In [ ]:
print(ndh)
print(ndh.shape)
print(8*14)

# extra

In [ ]:
keypoints_dataframe = pd.read_csv("DATA/Falls/Falls_Frames_annot.csv", index_col = 0)
labels_dataframe = pd.read_csv("DATA/Falls/Falls_Labels.csv", index_col = 0)

ndh = np.zeros((np.int64(len(keypoints_dataframe)/14),8*14))
ii = 43

for fall_number in range(8,9) : #3,13
    KP = keypoints_dataframe.loc['Fall_'+str(fall_number)][['KP','x','y','W','H','frame']]
    size = len(KP)
    
    keypoints = np.zeros((np.int64(size/14),14,2))
    W = np.zeros((np.int64(size/14)))
    H = np.zeros((np.int64(size/14)))
    
    for j in range(0,np.int64(size/14)):
        KP2 = KP.iloc[j*14:j*14+14]
        #print(labels_dataframe['label'].iloc[ii])
        keypoints[j,:,:] = KP2[['x','y']].to_numpy()
        W[j] = KP2['W'][0]
        H[j] = KP2['H'][0]
        
        ndh[ii,:] = NDH(keypoints[j,:,:],W[j],H[j],'RegKey')
        frame = KP2['frame'][0]
        frame = 'DATA/Falls/Fall_'+str(fall_number)+'/'+frame
        Display_img(frame,keypoints[j,:,:],labels_dataframe['label'].iloc[ii])
        ii += 1

In [ ]:
print(ndh[ii-13].reshape(8,14))